In [ ]:
import pandas as pd
import json
from enum import Enum

In [1]:
class FilingsQuery:
    """The FilingsQuery is a convenient helper interface allowing the specification of commonly used Filings query properties
       used to formulate a GraphQL expression used by the Filings API.
       
       To start:
       
       1. Create a Filings Query
          query = FilingsQuery()
          
       2. Use the fluent interface to define your properties.  For example:
          query.from('2021-02-12T00:00:00Z').to('2021-02-12T23:59:59Z')
          
       3. Generate the Graph QL Query
          graphQL = query.graphQL
    """
    # From date
    def start_date(self, date):
        """Defines a from date to use in our date range to search for filings
        """
        if date is None: 
            raise NameError('from date must exist')  
            
        self.__from = date
        return self
    
    # To date
    def end_date(self, date):
        """Defines a to date to use in our date range to search for filings
        """
        if date is None: 
            raise NameError('to date must exist')  
            
        self.__to = date
        return self    
    
    # Text to search
    def text(self, text):
        """Defines the text string to search across the Filings database
        """
        if text is None: 
            raise NameError('text must exist')  
            
        self.__text = text
        return self 
                  
    # Text to search
    def orgid(self, id):
        """Defines the Organization ID used to query across the Filings database
        """
        if id is None: 
            raise NameError('Org ID must exist')  
            
        self.__orgId = id
        return self
    
    # Form Type to search
    def form_type(self, form):
        """Defines the Form Type used to query across the Filings database
        """
        if form is None: 
            raise NameError('Form Type must exist')  
            
        self.__formType = form
        return self   
    
    def sections(self, section):
        """Defines the sections of data within the filings to retrieve
        """
        if section is None:
            raise NameError('Section must exist')
            
        if isinstance(section, list):
            for item in section:
                self.__sections.append(item)
        else:
            self.__sections.append(section)
            
        return self
    
    # Limit hits per response
    def limit(self, limit):
        """Defines the limit of document hits per response
        """
        if limit is None: 
            raise NameError('Limit must exist')  
            
        self.__limit = limit
        return self  
    
    # Feed to search
    def feed(self, feed):
        """Defines the Feed used to query across the Filings database
        """
        if not isinstance(feed, FilingsQuery.Feed): 
            raise NameError(f'**Invalid feed specified.  Feed must be: {FilingsQuery.Feed}\n\tEg: FilingsQuery.feed(FilingsQuery.Feed.Edgar)')
   
        self.__feedId = feed
        return self
    
    def high_level_categoryId(self, category):
        """Defines the High Level Category ID used to query across the Filings database
        """
        if category is None: 
            raise NameError('High Level category ID must exist')
   
        self.__highLevelId = category
        return self

    def mid_level_categoryId(self, category):
        """Defines the Mid Level Category ID used to query across the Filings database
        """
        if category is None: 
            raise NameError('Mid Level category ID must exist')
   
        self.__midLevelId = category
        return self
    
    def retrieve_text(self, retrieve_text):
        """Boolean to control whether to include text version of the document, if available, within the response body
           Default: False
        """
        if retrieve_text is None: 
            raise NameError('Boolean value must exist')  
            
        self.__retrieveText = retrieve_text
        return self    
  
    @property
    # graphQL
    # Extract the Filings graphQL query string based on the properties defined
    def graphQL(self):
        """Generate the Graph QL expression based on the defined properties
        """
        # Apply filters
        query = self.__template.replace("$filter", self.__apply_filters())
            
        # Apply keywords
        query = query.replace("$keywords", self.__apply_keywords())
        
        # Apply sections
        query = query.replace("$sections", self.__apply_sections())
        
        # Include Document Text within response?
        query = query.replace("$documenttext", self.__documentText if self.__retrieveText else "")
        
        # Limit document hits per response
        query = query.replace("$limit", str(self.__limit))
        
        # Sort Order
        return query.replace("$sortOrder", self.__sortOrder)
    
    def __apply_filters(self):
        filters = []        
        
        # Apply filing data range
        if not(self.__from is None or self.__to is None):
            filters.append(self.__dateFilter.replace("$fromdate", self.__from).replace("$todate", self.__to))
                           
        # Apply OrgID
        if not self.__orgId is None:
            filters.append(self.__orgFilter.replace("$orgId", self.__orgId))
            
        # Apply Form Type
        if not self.__formType is None:
            filters.append(self.__formTypeFilter.replace("$formtype", self.__formType))
            
        # Apply Feed ID
        if not self.__feedId is None:
            filters.append(self.__feedIdFilter.replace("$feedid", str(self.__feedId.value)))
            
        # Apply High Level Category ID
        if not self.__highLevelId is None:
            filters.append(self.__highLevelCategory.replace("$highlevelid", str(self.__highLevelId)))
            
        # Apply Mid Level Category ID
        if not self.__midLevelId is None:
            filters.append(self.__midLevelCategory.replace("$midlevelid", str(self.__midLevelId)))
            
        if len(filters) > 0:           
            filterExpression = ""
            for val in filters:
                if (len(filterExpression) > 0): filterExpression += ','
                filterExpression += val
                
            if len(filters) > 1:
                filterExpression = '{AND: [' + filterExpression + ']}'

            return self.__filter.replace("$filters", filterExpression)
        
        return ""  
    
    class Feed(Enum):
        EDGAR = 1
        CRIS = 2
        SEDAR = 4
        TANSHIN = 5
        YUHO = 6
        OBI = 7
        CHINA = 9
        DART = 11
        BRIDGE = 12
        SECNONEDGAR = 13
        ITALY = 15
        MUNI = 17
        
    
    def __apply_keywords(self):
        if not self.__text is None:
            return self.__keywords.replace("$text", self.__text)
        return ""
    
    def __apply_sections(self):
        contents = ""

        if len(self.__sections) > 0:
            for section in self.__sections:
                # For simplicity, I'm making the assumption that each section automatically uses the 'Text' property
                contents += f'{section} {{ Text }} '
            return self.__sectionsBlock.replace("$sectionStanza", contents)

        return contents
        
    # Instantiate a FilingsQuery object
    def __init__(self):        
        self.__from = None
        self.__to = None
        self.__text = None
        self.__orgId = None
        self.__formType = None
        self.__sortOrder = 'DESC'
        self.__limit = 10
        self.__retrieveText = False
        self.__feedId = None
        self.__highLevelId = None
        self.__midLevelId = None
        self.__sections = []
        
        # The basic GraphQL stanza
        self.__template = '{FinancialFiling($filter $keywords sort: {FilingDocument: {DocumentSummary: {FilingDate: $sortOrder}}}, limit: $limit ) {_metadata {totalCount cursor} FilingOrganization {Names {Name {OrganizationName(filter: {AND: [ {LanguageId_v2: {EQ: "505062"}}, {NameTypeCode: {EQ: "LNG"}}]}) {Name}}}} FilingDocument {Identifiers {OrganizationId Dcn} DocId FinancialFilingId $sections DocumentSummary {DocumentTitle FeedName FormType HighLevelCategory MidLevelCategory FilingDate SecAccessionNumber SizeInBytes} FilesMetaData {FileName MimeType} $documenttext}}}'

        # Supported filtering criteria
        # Define the filtering criteria, based on criteria such as the type of form, filing date.
        # The details of the filter will be defined under the 'FinancialFiling stanza.
        self.__filter = 'filter: $filters,'
        self.__dateFilter = '{FilingDocument: {DocumentSummary: {FilingDate: {BETWN: {FROM: "$fromdate", TO: "$todate"}}}}}'
        self.__orgFilter = '{FilingDocument: {Identifiers: {OrganizationId: {EQ: "$orgId"}}}}'
        self.__formTypeFilter = '{FilingDocument: {DocumentSummary: {FormType: {EQ: "$formtype"}}}}'
        self.__feedIdFilter = '{FilingDocument: {DocumentSummary: {FeedId: {EQ: "$feedid"}}}}'
        self.__midLevelCategory = '{FilingDocument: {DocumentSummary: {MidLevelCategoryId: {EQ: "$midlevelid"}}}}'
        self.__highLevelCategory = '{FilingDocument: {DocumentSummary: {HighLevelCategoryId: {EQ: "$highlevelid"}}}}'
        
        # Keywords to search
        # If a text keyword has been defined, include the following criteria within the search query expression
        # defined under the 'FinancialFiling' stanza.
        self.__keywords = 'keywords: {searchstring: "FinancialFiling.FilingDocument.DocumentText:$text"},'
        
        # Embed Document Text
        # If the following stanza is defined within the 'FilingDocument' section, the response will instruct the server to embed
        # The text version of the document within the response body.
        self.__documentText = 'DocumentText'
        
        # Sections block
        self.__sectionsBlock = 'Sections { $sectionStanza }'
                